In [1]:
# 데이터 활용에 필요한 기본 패키지 로딩
import sys #access to system parameters 
import pandas as pd
import sklearn #collection of machine learning algorithms
import numpy as np #foundational package for scientific computing
import tensorflow as tf
import keras
import os

Using TensorFlow backend.


In [0]:
#데이터 전처리 관련 라이브러리 로드
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import ShuffleSplit

# Deep Learning Model 로드
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation 
from keras.wrappers.scikit_learn import KerasClassifier

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

#HyperParameter Tuning을 위한 라이브러리 로드
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

#모델 평가를 위한 라이브러리 로드
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn import model_selection

#수학 & 통계 관련 라이브러리 로드
import scipy.stats as st
from collections import Counter

#Configure Visualization Defaults
#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Auth 인증 및 Google Drive 활용 Data load
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/gdrive', force_remount=True)

# Google Drive 내 Custom Class 경로 지정
import sys
sys.path.insert(0, '/gdrive/My Drive/CustomClasses')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [6]:
# !ls /gdrive/My\ Drive/kaggle
!ls /gdrive/My\ Drive/data

crawl-300d-2M.vec  glove.840B.300d.txt	  Study
dogs-vs-cats.zip   joongo.csv		  test.csv
FashionMNIST	   sample_submission.csv  train.csv


# **EDA**

필요한 Package Load 및 학습 데이터 Load

In [0]:
df_joongo = pd.read_csv('/gdrive/My Drive/data/joongo.csv', encoding='utf-8')

In [8]:
# 데이터 정상 Load 확인
df_joongo.head()

,NAME,YEAR,KM,PRICE,FUEL
0,아반떼 AD,2016-01-01,99249.0,1050.0,디젤
1,아반떼 AD,2015-12-01,75806.0,1070.0,가솔린
2,아반떼 AD,2016-01-01,22335.0,1080.0,가솔린
3,아반떼 AD,2016-01-01,87339.0,1090.0,디젤
4,아반떼 AD,2016-03-01,77861.0,1090.0,디젤


In [9]:
print(df_joongo.isnull().any())

NAME     False
YEAR     False
KM       False
PRICE    False
FUEL     False
dtype: bool


In [11]:
print("joongo null value counts are \n {}".format(df_joongo.isnull().sum()))
print("Train total length is ", len(df_joongo))

joongo null value counts are 
 NAME     0
YEAR     0
KM       0
PRICE    0
FUEL     0
dtype: int64
Train total length is  64


In [16]:
#however passing by reference is convenient, because we can clean both datasets at once
print('# Col Name: Unique Value / Total Size')
print("-"*25)

for idx, col in enumerate(df_joongo):    
    # Unique Variable Check for which columns can be one hot encoding
    print(col, ':', df_joongo[col].unique().size, '/', len(df_joongo[col]))  

# Col Name: Unique Value / Total Size
-------------------------
NAME : 7 / 64
YEAR : 44 / 64
KM : 64 / 64
PRICE : 55 / 64
FUEL : 2 / 64


In [18]:
print(df_joongo.columns)
print()


Index(['NAME', 'YEAR', 'KM', 'PRICE', 'FUEL'], dtype='object')

In [22]:
# 데이터들의 기초통계량 체크
for col in df_joongo:    
    # dataset 수치형 변수 살펴보기
    dataset_n_cols = [col for col in df_joongo.columns if df_joongo[col].dtype not in ['O']]
    dataset_c_cols = [col for col in df_joongo.columns if df_joongo[col].dtype in ['O']]
    
print('## Description of Numeric Values in','{}'.format('joongo dataframe'))
print(df_joongo[dataset_n_cols].describe())
print("-"*50)
print('## Description of Categorical Values in','{}'.format('joongo dataframe'))
print(df_joongo[dataset_c_cols].describe())

## Description of Numeric Values in joongo dataframe
                  KM        PRICE
count      64.000000    64.000000
mean    70446.078125  1431.093750
std     35005.054878   675.571756
min     11705.000000   580.000000
25%     39473.500000  1047.500000
50%     74433.500000  1325.000000
75%     95010.000000  1635.000000
max    137925.000000  4190.000000
--------------------------------------------------
## Description of Categorical Values in joongo dataframe
          NAME        YEAR FUEL
count       64          64   64
unique       7          44    2
top     아반떼 AD  2016-01-01   디젤
freq        16           5   34
